# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file='../Output/CityWeatherData.csv'
CityVacation_df=pd.read_csv(file)
CityVacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,77.72,73,44,19.62,PF,1632174982
1,Batemans Bay,-35.7167,150.1833,50.13,50,12,7.76,AU,1632174983
2,Mataura,-46.1927,168.8643,45.37,97,100,0.96,NZ,1632174983
3,Hithadhoo,-0.6000,73.0833,82.99,73,84,11.45,MV,1632174983
4,Nikolskoye,59.7035,30.7861,44.49,73,97,7.65,RU,1632174984


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [14]:
fig = gmaps.figure(center=(0,0),zoom_level=2)
heatmap_layer =gmaps.heatmap_layer(CityVacation_df[['Lat','Lng']],weights=CityVacation_df['Humidity'],max_intensity=100, point_radius=6.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
len(CityVacation_df)

585

In [18]:
#Drop any rows will null values
NewCityVacation=CityVacation_df.dropna(how='any')
len(NewCityVacation)

581

In [19]:
NewCityVacation.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [28]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

idealCity=NewCityVacation.loc[(NewCityVacation['Max Temp']<=80) & (NewCityVacation['Max Temp']>=70)]
idealCity=idealCity.loc[(idealCity['Wind Speed']<=10)]
idealCity=idealCity.loc[(idealCity['Cloudiness']==0)]
idealCity

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
271,Nara,34.6851,135.8049,72.72,77,0,6.04,JP,1632175110
327,Kiryat Gat,31.6100,34.7642,79.48,68,0,4.56,IL,1632174904
460,Namwala,-15.7504,26.4384,70.45,32,0,4.45,ZM,1632175194
474,Olinda,-8.0089,-34.8553,76.73,61,0,9.22,BR,1632175202
505,Semnan,35.5729,53.3971,76.21,22,0,5.23,IR,1632175219
519,Morro Bay,35.3658,-120.8499,75.69,48,0,4.61,US,1632175064


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df=idealCity
hotel_df['Hotel Name']=''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
271,Nara,34.6851,135.8049,72.72,77,0,6.04,JP,1632175110,
327,Kiryat Gat,31.6100,34.7642,79.48,68,0,4.56,IL,1632174904,
460,Namwala,-15.7504,26.4384,70.45,32,0,4.45,ZM,1632175194,
474,Olinda,-8.0089,-34.8553,76.73,61,0,9.22,BR,1632175202,
505,Semnan,35.5729,53.3971,76.21,22,0,5.23,IR,1632175219,
519,Morro Bay,35.3658,-120.8499,75.69,48,0,4.61,US,1632175064,


In [37]:
for index,row in hotel_df.iterrows():
    try:
        url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params={'keyword':'hotel','radius':5000,'key':g_key}
                
        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"
        response=requests.get(url,params=params)   
        get_hotel=response.json()
        hotel_df.loc[index, "Hotel Name"] = get_hotel["results"][0]["name"]
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
271,Nara,34.6851,135.8049,72.72,77,0,6.04,JP,1632175110,Hotel Nikko Nara
327,Kiryat Gat,31.6100,34.7642,79.48,68,0,4.56,IL,1632174904,Desert Gat
460,Namwala,-15.7504,26.4384,70.45,32,0,4.45,ZM,1632175194,Karibu Guest House
474,Olinda,-8.0089,-34.8553,76.73,61,0,9.22,BR,1632175202,Hotel 7 Colinas
505,Semnan,35.5729,53.3971,76.21,22,0,5.23,IR,1632175219,Semnan Traditional Hotel
519,Morro Bay,35.3658,-120.8499,75.69,48,0,4.61,US,1632175064,Inn at Morro Bay


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))